In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
import string
import ast
import re

import os

In [8]:
# functions
def is_county(text):
    county_keywords = ["county", "Parish", "parish"]
    if any(keyword in text for keyword in county_keywords):
        return True
    else:
        return False

def get_city_name(text):
    delete_words = []
    for word in delete_words:
        text = text.replace(word,  "")
    text = text.replace("-", " ")
    text = re.sub(r' \([^()]*\)', '', text)
    text = text.replace("St. ", "saint ")
    text = text.replace("St ", "saint ")
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text
    
def get_county_name(text):
    delete_words = [" (county)", " County", " county", " (County)", " Parish", " parish"]
    for word in delete_words:
        text = text.replace(word,  "")
    text = text.replace("St. ", "saint ")
    text = text.replace("St ", "saint ")
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def load_data(DATA_DIR):
    data = {}
    # load data
    data["gun"] = pd.read_csv(os.path.join(DATA_DIR, "gun-violence-data_01-2013_03-2018.csv"))
    print(f"Gun violence data loaded. {data['gun'].shape}")
    data["cities"] = pd.read_csv(os.path.join(DATA_DIR, "us_cities2counties.csv"))
    print(f"City to county data loaded. {data['cities'].shape}")
    data["pol"] = pd.read_csv(os.path.join(DATA_DIR, "us_counties_election2016.csv"))
    print(f"2016 Election data loaded. {data['pol'].shape}")
    data["demo"] = pd.read_csv(os.path.join(DATA_DIR, "cc-est2018-alldata.csv"), encoding='latin-1')
    print(f"Demographics data loaded. {data['demo'].shape}")
    data["edu"] = pd.read_csv(os.path.join(DATA_DIR, "us_counties_education.tsv"), sep='\t')
    print(f"County level education data loaded. {data['edu'].shape}")
    data["pov"] = pd.read_csv(os.path.join(DATA_DIR, "us_counties_poverty.tsv"), sep='\t')
    print(f"County level poverty data loaded. {data['pov'].shape}")
    data["unem"] = pd.read_csv(os.path.join(DATA_DIR, "us_counties_unemployment.tsv"), sep='\t')
    print(f"County level unemployment data loaded. {data['unem'].shape}")
    return data
    
def process_gun_violence_data(gun, cities):
    print(f"Processing {gun.shape[0]} gun violence incidents from 2010-2018.")
    # get subset of columns
    gun = gun[["incident_id",
               "date",
               "state",
               "city_or_county",
               "n_killed",
               "n_injured",
               "gun_stolen",
               "gun_type"]]
    # get dates
    gun['Year'] = gun['date'].apply(lambda datestring: int(datestring[0:4]))
    gun['Month'] = gun['date'].apply(lambda datestring: int(datestring[6:7]))
    gun['Day'] = gun['date'].apply(lambda datestring: int(datestring[9:10]))
    # fill nans
    gun["gun_stolen"] = gun["gun_stolen"].fillna("No Info")
    gun["gun_type"] = gun["gun_type"].fillna("No Info")
    # get binary labels for stolen gun used, fatal, and mass shooting
    gun['StolenGunUsed'] = gun["gun_stolen"].apply(lambda x: 1 if "::Stolen" in x else 0)
    gun['MassShooting'] = gun["n_killed"].apply(lambda x: 1 if x >= 5 else 0)
    # get binary labels for AR shooting
    AR_keywords = ["AR-15", "AK-47"]
    gun['ARShooting'] = gun["gun_type"].apply(lambda x: 1 if any(keyword in x for keyword in AR_keywords) else 0)
    # rename columns
    gun = gun.rename({'state': 'State'}, axis=1)
    
    # separate into city and county based incidents
    print("Separating incidents that have county or city names.")
    gun["isCounty"] = gun["city_or_county"].apply(lambda x: is_county(x))
    gun_county = gun[gun["isCounty"] == True]
    gun_city = gun[gun["isCounty"] == False]
    gun_county = gun_county.drop("isCounty", axis=1)
    gun_city = gun_city.drop("isCounty", axis=1)
    print(f"{gun_city.shape[0]} incidents with city info.")
    print(f"{gun_county.shape[0]} incidents with county info.")

    # process county names
    gun_county['County'] = gun_county['city_or_county'].apply(lambda x: get_county_name(x))
    gun_county["City"] = None
    # process city names
    gun_city['City'] = gun_city['city_or_county'].apply(lambda x: get_city_name(x))
    
    # process cities2counties data
    cities = process_cities2counties_data(cities)
    
    # join with gun violence data that do not have counties
    print(f"Joining gun violence incidents that do not have county names with cities2counties dataset.")
    gun_city = gun_city.merge(cities, how="left", on=["State", "City"])
    print(f"Unable to find corresponding counties for {gun_city['County'].isna().sum()} incidents.")
    
    # join with gun violence data that do have counties to get FIPS
    print("Joining gun violence incidents that have county names to get FIPS")
    gun_county = gun_county.merge(cities[["State", "County", "CountyFIPS"]], how="left", on=["State", "County"])
    print(f"Unable to find FIPS for {gun_county[gun_county['CountyFIPS'].isna()].shape[0]} incidents.")
    
    # join the two datasets
    print("Joining the county and city based incidents back into one table.")
    gun = pd.concat([gun_county, gun_city])
    gun = gun[["incident_id",
                "Year",
                "Month",
                "Day",
                "State",
                "City",
                "County",
                "CountyFIPS",
                "n_killed",
                "n_injured",
                "MassShooting",
                "StolenGunUsed",
                "ARShooting"]]
    print(f"All gun violence incidents dataset size:{gun.shape}")
    # drop rows that have nans for CountyFIPS and duplicates
    gun = gun[gun['CountyFIPS'].notna()]
    gun = gun.drop_duplicates()
    print(f"Final gun violence incidents dataset size:{gun.shape}")
    return gun

def process_cities2counties_data(cities):
    # expand contractions
    cities['county_name'] = cities['county_name'].apply(lambda x: get_county_name(x))
    cities['city'] = cities['city'].apply(lambda x: get_city_name(x))

    # Get city-state key to join on 
    cities['County'] = cities['county_name']
    uscc = cities[["state_name", "city", "County", "county_fips"]]
    uscc = uscc.rename({"state_name": "State",
                        "city": "City",
                        "county_fips": "CountyFIPS"}, axis=1)
    uscc["CountyFIPS"] = uscc["CountyFIPS"].apply(lambda x: f"{int(x):05}")
    return uscc

def process_election_data(pol):
    # get PVI for 2016 election
    pol["PVI_2016"] = pol['per_point_diff'].apply(lambda x: float(x[:-1]))
    pol["PVI_2016"] = pol.apply(lambda x: x["PVI_2016"] if x["per_dem"] > x["per_gop"] else x["PVI_2016"]*-1, axis=1)
    pol["County"] = pol["county_name"].apply(lambda x: get_county_name(x))
    # reading the data from the file
    with open(os.path.join(DATA_DIR, "us_state_abbrev")) as f:
        state_abbrev = f.read()
    # reconstructing the data as a dictionary
    state_abbrev = ast.literal_eval(state_abbrev)
    abbrev_us_state = dict(map(reversed, state_abbrev.items()))
    # get full state names and district numbers
    pol['State'] = pol['state_abbr'].apply(lambda x: abbrev_us_state[x[0:2]])
    # drop unnecessary columns
    pol = pol[["State", "County", "combined_fips", "PVI_2016"]]
    pol = pol.rename({"combined_fips": "CountyFIPS"}, axis=1)
    pol["CountyFIPS"] = pol["CountyFIPS"].apply(lambda x: f"{x:05}")
    return pol

def process_education_data(edu):
    edu = edu.dropna()
    edu = edu.rename({"Percent of adults with less than a high school diploma 2015-19": "Edu_perc_NoHS",
                      "Percent of adults with a high school diploma only 2015-19": "Edu_perc_HSOnly",
                      "Percent of adults completing some college or associates degree 2015-19": "Edu_perc_SomeCollege",
                      "Percent of adults with a bachelor's degree or higher 2015-19": "Edu_perc_College"}, axis=1)
    edu["CountyFIPS"] = edu["FIPS Code"].apply(lambda x: f"{int(x):05}")
    # convert percentage to fraction
#     edu["Edu_frac_NoHS"] = edu["Edu_perc_NoHS"]/100
#     edu["Edu_frac_HSOnly"] = edu["Edu_perc_HSOnly"]/100
#     edu["Edu_frac_SomeCollege"] = edu["Edu_perc_SomeCollege"]/100
#     edu["Edu_frac_College"] = edu["Edu_perc_College"]/100
    edu = edu[["CountyFIPS", "Edu_perc_NoHS", "Edu_perc_HSOnly", "Edu_perc_SomeCollege", "Edu_perc_College"]]
    return edu

def process_poverty_data(pov):
    pov["FIPStxt"] = pov["FIPStxt"].apply(lambda x: f"{x:05}")
    pov = pov[["FIPStxt", "PCTPOVALL_2019"]]
    pov = pov.rename({"FIPStxt": "CountyFIPS",
                      "PCTPOVALL_2019": "Econ_perc_poverty"}, axis=1)
    # convert percentage to fraction
#     pov["Econ_frac_poverty"] = pov["Econ_perc_poverty"]/100
    return pov

def process_demographics_data(demo, year):
    # get CountyFIPS for key
    demo["STATE"] = demo["STATE"].apply(lambda x: f"{int(x):02}")
    demo["COUNTY"] = demo["COUNTY"].apply(lambda x: f"{int(x):03}")
    demo["CountyFIPS"] = demo["STATE"] + demo["COUNTY"]
    # get year of demographics data
    year_mapping = {1:2010,
                    2:2010,
                    3:2010,
                    4:2011,
                    5:2012,
                    6:2013,
                    7:2014,
                    8:2015,
                    9:2016,
                    10:2017,
                    11:2018,
                    12:2019}
    demo["YEAR"] = demo["YEAR"].replace(year_mapping)
    # filter to desired year
    demo = demo[demo['YEAR'] == year]
    # filter to all ages
    demo = demo[demo["AGEGRP"] == 0]
    # filter down to desired columns
    demo = demo[["YEAR",
                 "STNAME",
                 "CTYNAME",
                 'CountyFIPS',
                 "TOT_POP",
                 "NHWA_MALE",
                 "NHWA_FEMALE", 
                 "NHBA_MALE",
                 "NHBA_FEMALE",
                 "NHIA_MALE",
                 "NHIA_FEMALE",
                 "NHAA_MALE",
                 "NHAA_FEMALE",
                 "NHNA_MALE",
                 "NHNA_FEMALE", 
                 "H_MALE",
                 "H_FEMALE"]]
    # get fraction of population in each demo group
    demo['perc_NHWA'] = (demo["NHWA_MALE"] + demo["NHWA_FEMALE"])/demo["TOT_POP"]*100
    demo['perc_NHBA'] = (demo["NHBA_MALE"] + demo["NHBA_FEMALE"])/demo["TOT_POP"]*100
    demo['perc_NHIA'] = (demo["NHIA_MALE"] + demo["NHIA_FEMALE"])/demo["TOT_POP"]*100
    demo['perc_NHAA'] = (demo["NHAA_MALE"] + demo["NHAA_FEMALE"])/demo["TOT_POP"]*100
    demo['perc_NHNA'] = (demo["NHNA_MALE"] + demo["NHNA_FEMALE"])/demo["TOT_POP"]*100
    demo['perc_HISP'] = (demo["H_MALE"] + demo["H_FEMALE"])/demo["TOT_POP"]*100
    # filter down to demo percentages
    demo = demo[["YEAR",
                 "STNAME",
                 "CTYNAME",
                 'CountyFIPS',
                 "TOT_POP",
                 'perc_NHWA',
                 'perc_NHBA',
                 'perc_NHIA',
                 'perc_NHNA',
                 'perc_HISP']]
    return demo
    
def process_unemployment_data(unem, year):
    unem = pd.read_csv("data/us_counties_unemployment.tsv", sep='\t')
    unem["CountyFIPS"] = unem["FIPS_Code"].apply(lambda x: f"{x:05}")
    unem = unem[["CountyFIPS", f"Unemployment_rate_{year}"]]
    return unem
    

In [14]:
# inputs
DATA_DIR = "data/raw"

# load data
data = load_data(DATA_DIR)

Gun violence data loaded. (239677, 29)
City to county data loaded. (28338, 17)
2016 Election data loaded. (3141, 11)
Demographics data loaded. (656678, 80)
County level education data loaded. (3288, 7)
County level poverty data loaded. (3193, 34)
County level unemployment data loaded. (3275, 49)


In [10]:
# process gun violence incidents data
data["gun"] = process_gun_violence_data(data["gun"], data["cities"])

Processing 239677 gun violence incidents from 2010-2018.


/home/ryan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

Separating incidents that have county or city names.
235721 incidents with city info.
3956 incidents with county info.
Joining gun violence incidents that do not have county names with cities2counties dataset.
Unable to find corresponding counties for 12622 incidents.
Joining gun violence incidents that have county names to get FIPS
Unable to find FIPS for 25 incidents.
Joining the county and city based incidents back into one table.

/home/ryan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:108: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




All gun violence incidents dataset size:(299811, 13)
Final gun violence incidents dataset size:(227090, 13)


In [11]:
# pick years
years = [2014, 2015, 2016, 2017]
complete_data = {}

# process other datasets
pol = process_election_data(data["pol"])
edu = process_education_data(data["edu"])
pov = process_poverty_data(data["pov"])

# loop through each year and generate datasets
for year in years:
    # load year specific datasets
    unem = process_unemployment_data(data["unem"], year)
    demo = process_demographics_data(data["demo"], year)
    
    # get subset of gun violence incidents corresponding to the desired year
    gun = data["gun"][data["gun"]["Year"] == year]
    print(f"{gun.shape[0]} gun violence incidents found in {year}.")
    # group by CountyFIPS
    gun_grouped = gun.groupby(["CountyFIPS", "County", "State"])[["n_killed",
                                                                   "n_injured",
                                                                   "MassShooting",
                                                                   "StolenGunUsed",
                                                                   "ARShooting"]].sum().reset_index()
    gun_grouped = gun_grouped.sort_values(by="n_killed", ascending=False)
    gun_grouped_incs = gun.groupby(["CountyFIPS", "County", "State"])["incident_id"].count().to_frame().reset_index()
    gun_grouped_incs = gun_grouped_incs.rename({"incident_id": "n_shootings"}, axis=1)
    gun_grouped = gun_grouped.merge(gun_grouped_incs[["CountyFIPS", "n_shootings"]], how="left", on="CountyFIPS")
    # join with demo dataset
    gun_grouped = gun_grouped.merge(demo[["CTYNAME", 
                                          "CountyFIPS", 
                                          "TOT_POP", 
                                          'perc_NHWA',
                                          'perc_NHBA',
                                          'perc_NHIA',
                                          'perc_NHNA',
                                          'perc_HISP']], how="left", on="CountyFIPS")
    # join with 2016 elections datatset
    gun_grouped = gun_grouped.merge(pol[['CountyFIPS', "PVI_2016"]], how='left', on='CountyFIPS')
    # join with poverty, education, and unemployment datasets
    gun_grouped = gun_grouped.merge(edu, how="left", on="CountyFIPS")
    gun_grouped = gun_grouped.merge(pov, how="left", on="CountyFIPS")
    gun_grouped = gun_grouped.merge(unem, how="left", on="CountyFIPS")
    # get population normalized rates
    gun_grouped["shootings_per_100k"] = gun_grouped["n_shootings"]/gun_grouped["TOT_POP"]*1e5
    gun_grouped["killed_per_100k"] = gun_grouped["n_killed"]/gun_grouped["TOT_POP"]*1e5
    gun_grouped["injured_per_100k"] = gun_grouped["n_injured"]/gun_grouped["TOT_POP"]*1e5
    gun_grouped["mass_shootings_per_100k"] = gun_grouped["MassShooting"]/gun_grouped["TOT_POP"]*1e5
    gun_grouped["AR_shooting_per_100k"] = gun_grouped["ARShooting"]/gun_grouped["TOT_POP"]*1e5
    # change unemployment column name
    gun_grouped = gun_grouped.rename(index=str, columns={f"Unemployment_rate_{year}": "Unemployment_rate"})
    # drop rows with NAs
    n_rows = gun_grouped.shape[0]
    gun_grouped = gun_grouped.dropna()
    print(f"Dropped {n_rows - gun_grouped.shape[0]} counties due to NAN.")
    # save completed data
    gun_grouped.to_csv(os.path.join(DATA_DIR, "complete", f"GunViolenceByCounty_{year}.csv"), index=False)

47265 gun violence incidents found in 2014.
Dropped 1 counties due to NAN.
50825 gun violence incidents found in 2015.
Dropped 1 counties due to NAN.
56508 gun violence incidents found in 2016.
Dropped 2 counties due to NAN.
58978 gun violence incidents found in 2017.
Dropped 2 counties due to NAN.


In [12]:
gun_grouped.head()

,CountyFIPS,County,State,n_killed,n_injured,MassShooting,StolenGunUsed,ARShooting,n_shootings,CTYNAME,...,Edu_perc_HSOnly,Edu_perc_SomeCollege,Edu_perc_College,Econ_perc_poverty,Unemployment_rate,shootings_per_100k,killed_per_100k,injured_per_100k,mass_shootings_per_100k,AR_shooting_per_100k
0,17031,cook,Illinois,722,3044,0,12,13,3272,Cook County,...,23.1,25.2,38.8,13.0,5.2,62.868647,13.872605,58.487825,0.00000,0.249784
1,06037,los angeles,California,387,347,0,4,9,756,Los Angeles County,...,20.6,26.0,32.5,13.4,4.8,7.471272,3.824580,3.429274,0.00000,0.088944
2,48201,harris,Texas,305,438,1,5,5,642,Harris County,...,23.4,26.6,31.5,15.0,5.1,13.764539,6.539228,9.390760,0.02144,0.107200
3,24510,baltimore,Maryland,304,655,0,21,1,1117,Baltimore city,...,29.1,24.2,31.9,20.4,6.1,183.162496,49.849059,107.405045,0.00000,0.163977
4,29510,saint louis,Missouri,300,514,0,5,4,737,St. Louis city,...,23.9,27.5,36.3,20.4,4.5,239.389864,97.444992,166.955753,0.00000,1.299267


In [13]:
gun_grouped.columns

Index(['CountyFIPS', 'County', 'State', 'n_killed', 'n_injured',
       'MassShooting', 'StolenGunUsed', 'ARShooting', 'n_shootings', 'CTYNAME',
       'TOT_POP', 'perc_NHWA', 'perc_NHBA', 'perc_NHIA', 'perc_NHNA',
       'perc_HISP', 'PVI_2016', 'Edu_perc_NoHS', 'Edu_perc_HSOnly',
       'Edu_perc_SomeCollege', 'Edu_perc_College', 'Econ_perc_poverty',
       'Unemployment_rate', 'shootings_per_100k', 'killed_per_100k',
       'injured_per_100k', 'mass_shootings_per_100k', 'AR_shooting_per_100k'],
      dtype='object')